# BBS with ECDSA Holder Binding

While BBS allows to present parts of the Verifiable Credential in an anonymous
and unlinkable manner, there are limits.
The [RFC](https://datatracker.ietf.org/doc/draft-irtf-cfrg-bbs-signatures/) only
supports selective disclosure.
This works for fields which have a big anonymity set, like the NPA, or the canton of
residence.
However, for more linkable fields, like the birthdate, full name, the anonymity set
gets very small and can drop to 1.
An anonymity set of 1 means that there is only one VC with that field, so it can
be linked between usages.

To achieve holder binding, it is necessary to sign each presentation of a VC
with the private key of the device.
For the verification part, the verifier usually needs the public key.
However, this public key is unique.
Even worse, the holder binding needs to be done for every presentation of the VC,
so all presentations can be linked!

One way to solve this is to create many VCs for a user, each VC with a corresponding
public key created by the Secure Element of the wallet, e.g., the phone.
This is not an ideal solution for the following reasons: 

- it increases load on the server who has to issue VCs on a regular basis
- to increase privacy, these must be renewed, even if the VCs are not used
- some secure elements have a limit of how many private keys they can store

Several other solutions have been proposed so far:

- BBS# from Orange - which is missing a scientific paper
[Presentation](https://csrc.nist.gov/csrc/media/presentations/2024/wpec2024-3b3/images-media/wpec2024-3b3-slides-antoine-jacques--BBS-sharp-eIDAS2.pdf)
- Google ZKP, presented at Funke 24 - but in June '25 there is still no code available
- Ubique proposed a solution based on [ZKAttest](https://eprint.iacr.org/2021/1183) - with code available, and based on a peer-reviewed paper from Cloudflare

## ZKAttest from Cloudflare

The ZKAttest paper from Cloudflare presents an interesting solution to create ZKPs
on operations for the P-256 curve.
In short, they use a new elliptic curve with properties allowing it to create short
proofs on operations on the P-256 curve.
This new curve is introduced in the ZKAttest paper and called `T-256`, or `Tom-256`.
While the solution from Google is very generic and works even with an example
using mDoc, the `T-256` curve is only useful if the VC is based on BBS(+).

As with SD-JWT, the holder gets their VC from the issuer:

1. The holder proves their identity, creates a keypair with their Secure Element, and sends the public key to the issuer
2. The issuer creates a VC with all value of the fields hashed, including the public key from the holder
3. The issuer signs the VC using the its secret key, and sends it to the holder
4. The holder stores this VC in their device

Unlike the SD-JWT, the BBS VC can be 'blinded', but still be verified using the public
key from the issuer.

In [ ]:
:dep ./ecdsa

println!("{}", ecdsa::add(2, 3));